负责把vnpy格式的1min csv合成为vnpy格式的日线 csv

In [1]:
import pandas as pd
import glob
import os
import time

In [2]:
data_folder_path = r"E:\mgc_1min"
data_path_list = glob.glob(os.path.join(data_folder_path, "*.csv"))
data_path_list = [path for path in data_path_list if "#"not in path]

In [3]:
day_bar_data_folder = r"E:\mgc_day"

In [4]:
len(data_path_list)

61

In [6]:
for number, path in enumerate(data_path_list):
    start_time = time.time()
    print("processing file {}/{}".format(number, len(data_path_list)))
    # read one min bar data
    frame = pd.read_csv(path)
    datetime_format = '%Y%m%d %H:%M:%S'
    frame.loc[:, 'datetime'] = pd.to_datetime(frame['datetime'], format=datetime_format)
    frame.loc[:, 'date'] = frame.loc[:, 'datetime'].apply(lambda x: x.date())
    float_columns = ['open', 'high', 'low', 'close', 'volume', 'open_interest']
    for col in float_columns:
        frame.loc[:, col] = frame[col].astype('float')
    frame = frame.sort_values(by="datetime")
    # build day bar data
    day_bar_frame = pd.DataFrame(columns=['open',
                                          'high',
                                          'low',
                                          'close',
                                          'volume',
                                          'date',
                                          'exchange',
                                          'open_interest',
                                          'interval',
                                          'symbol' ])
    for x, sub_data in frame.groupby("date"):
        new_row = pd.DataFrame(columns=['open',
                                        'high',
                                        'low',
                                        'close',
                                        'volume',
                                        'date',
                                        'exchange',
                                        'open_interest',
                                        'interval',
                                        'symbol' ])
        new_row.loc[0,"open"] = sub_data.iloc[0,:].loc["open"]
        new_row.loc[0,"high"] = sub_data.loc[:, "high"].max()
        new_row.loc[0,"low"] = sub_data.loc[:, "low"].min()
        new_row.loc[0,"close"] = sub_data.iloc[len(sub_data)-1,:].loc["close"]
        new_row.loc[0,"volume"] = sub_data.loc[:, "volume"].sum()
        new_row.loc[0,"date"] = x
        new_row.loc[0,"exchange"] = sub_data.iloc[0,:].loc["exchange"]
        new_row.loc[0,"open_interest"] = 0.0
        new_row.loc[0,"interval"] = sub_data.iloc[0,:].loc["interval"]
        new_row.loc[0,"symbol"] = sub_data.iloc[0,:].loc["symbol"]
        day_bar_frame = pd.concat([day_bar_frame, new_row])

    base_name = os.path.basename(path)
    day_bar_path = os.path.join(day_bar_data_folder, base_name)
    day_bar_frame.to_csv(day_bar_path, index=False)
    end_time = time.time()
    print("time used: {} sec".format((end_time - start_time)))

processing file 0/61
time used: 1.168595552444458 sec
processing file 1/61
time used: 2.278841972351074 sec
processing file 2/61
time used: 2.673124313354492 sec
processing file 3/61
time used: 2.3859963417053223 sec
processing file 4/61
time used: 2.236483097076416 sec
processing file 5/61
time used: 2.6917154788970947 sec
processing file 6/61
time used: 3.3183815479278564 sec
processing file 7/61
time used: 2.8776345252990723 sec
processing file 8/61
time used: 3.6446056365966797 sec
processing file 9/61
time used: 3.5353167057037354 sec
processing file 10/61
time used: 1.4722986221313477 sec
processing file 11/61
time used: 2.3308892250061035 sec
processing file 12/61
time used: 3.137504816055298 sec
processing file 13/61
time used: 2.734480381011963 sec
processing file 14/61
time used: 2.079603672027588 sec
processing file 15/61
time used: 2.680549144744873 sec
processing file 16/61
time used: 3.190436363220215 sec
processing file 17/61
time used: 3.115516185760498 sec
processing f